Comparison of White Island Airborne S02 and MDOAS Observations
--

**Purpose**

To compare White Island airborne S02 gas flux observations with those measured using the mini-DOAS instruments on the island. 

**Summary**

- Earliest MDOAS data in FITS is 2012-10-01. Filter airborne data to include only data starting a month before.
- Comparison only with observations from the same NZST day.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

**Import modules and setup parameters**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

names = ['dt', 'obs', 'err']
mdoasstart = '2012-09-01'

In [ ]:
def bardata(dfd):
    '''calculate dataframes for min, max, range, mean, q25, q50(median), and q75'''
    #range
    dfmin = dfd['obs'].groupby(dfd['date']).min()
    dfmin = dfmin.to_frame()
    dfmin.index = pd.to_datetime(dfmin.index)
    dfmax = dfd['obs'].groupby(dfd['date']).max()
    dfmax = dfmax.to_frame()
    dfmax.index = pd.to_datetime(dfmax.index)
    dfrange = dfmax-dfmin
    dfrange.index = pd.to_datetime(dfrange.index)
    #mean
    dfmean = dfd['obs'].groupby(dfd['date']).mean()
    dfmean = dfmean.to_frame()
    dfmean.index = pd.to_datetime(dfmean.index)
    #25th, 50th(median), and 75th percentile
    df25 = dfd['obs'].groupby(dfd['date']).quantile(0.25)
    df50 = dfd['obs'].groupby(dfd['date']).quantile(0.50)
    df75 = dfd['obs'].groupby(dfd['date']).quantile(0.75)
    return dfmin, dfmax, dfrange, dfmean, df25, df50, df75

In [ ]:
def axmax(ax):
    '''find axis maximum, across both x and y'''
    (ymin, ymax) = ax.get_ylim()
    (xmin, xmax) = ax.get_xlim()
    axmax = max(xmax,ymax)
    return axmax

**Retrieve data**

In [ ]:
#airborne cospec
url = 'https://fits.geonet.org.nz/observation?siteID=WI000&typeID=SO2-flux-a&methodID=cosp'
cosp = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#airborne contouring
url = 'https://fits.geonet.org.nz/observation?siteID=WI000&typeID=SO2-flux-a&methodID=cont'
cont = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#mdoas north-east point, assumed height
url = 'https://fits.geonet.org.nz/observation?typeID=SO2-flux-a&methodID=mdoas-ah&siteID=WI301'
nea = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#mdoas north-east point, calculated height
url = 'https://fits.geonet.org.nz/observation?typeID=SO2-flux-a&methodID=mdoas-ch&siteID=WI301'
nec = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#mdoas south rim, assumed height
url = 'https://fits.geonet.org.nz/observation?typeID=SO2-flux-a&methodID=mdoas-ah&siteID=WI302'
sra = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#mdoas south rim, calculated height
url = 'https://fits.geonet.org.nz/observation?typeID=SO2-flux-a&methodID=mdoas-ch&siteID=WI302'
src = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

**Trim airborne data to match observation period of MDOAS data**

In [ ]:
cosp = cosp[cosp.index > mdoasstart]
cont = cont[cont.index > mdoasstart]

**Convert MDOAS observations to NZST (this makes comparison easier)**

In [ ]:
nea.index = nea.index + pd.DateOffset(hours=12)
nec.index = nec.index + pd.DateOffset(hours=12)
sra.index = sra.index + pd.DateOffset(hours=12)
src.index = src.index + pd.DateOffset(hours=12)

**Make column in MDOAS dataframes containing date (no time)**

In [ ]:
nea['date'] = pd.DatetimeIndex(nea.index).normalize()
nec['date'] = pd.DatetimeIndex(nec.index).normalize()
sra['date'] = pd.DatetimeIndex(sra.index).normalize()
src['date'] = pd.DatetimeIndex(src.index).normalize()

**MDOAS on same day as airborne data**

In [ ]:
nead = nea[nea.date.isin(cosp.index)]
necd = nec[nec.date.isin(cosp.index)]
srad = sra[sra.date.isin(cosp.index)]
srcd = src[src.date.isin(cosp.index)]

**Airborne on same day as MDOAS data (so can cross-plot)**


In [ ]:
cospnea = cosp[cosp.index.isin(nead.date)]
cospnec = cosp[cosp.index.isin(necd.date)]
contnea = cont[cont.index.isin(nead.date)]
contnec = cont[cont.index.isin(necd.date)]

cospsra = cosp[cosp.index.isin(srad.date)]
cospsrc = cosp[cosp.index.isin(srcd.date)]
contsra = cont[cont.index.isin(srad.date)]
contsrc = cont[cont.index.isin(srcd.date)]

**Data for time-series boxplot**

In [ ]:
neamin, neamax, nearange, neamean, nea25, nea50, nea75 = bardata(nead)
necmin, necmax, necrange, necmean, nec25, nec50, nec75 = bardata(necd)
sramin, sramax, srarange, sramean, sra25, sra50, sra75 = bardata(srad)
srcmin, srcmax, srcrange, srcmean, src25, src50, src75 = bardata(srcd)

Time-series Comparison plots
--

In [ ]:
xsize = 15
ysize = 5

** North-East Point COSPEC**

In [ ]:
#import datetime
#datemax = datetime.datetime.now()
#datemin = datetime.datetime(2015,4,1)



In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2,ncols=1, sharex=True, figsize=(xsize, ysize))

#assumed height nea
ax0.plot(cosp['obs'], marker='o', linestyle=':', color='blue', label='cospec')
ax0.bar(nearange.index, nearange['obs'], bottom=neamin['obs'], width=0.01, color='red', edgecolor='red', alpha=0.25, align='center', label='mDOAS range')
ax0.plot(nea50.index, nea50, marker='o', markersize=3, color='black', linestyle='None', label='mDOAS median')
ax0.bar(nearange.index, nea75-nea25, bottom=nea25, width=3, color='red', edgecolor='red', align='center', label='mDOAS 25th/75th %')
ax0.set_title('North-East Point, COSPEC + mDOAS assumed height')
ax0.legend(loc='best', fontsize=8)

#calculated height nec
ax1.plot(cosp['obs'], marker='o', linestyle=':', color='blue', label='cospec')
ax1.bar(necrange.index, necrange['obs'], bottom=necmin['obs'], width=0.01, color='red', edgecolor='red', alpha=0.25, align='center', label='mDOAS range')
ax1.plot(nec50.index, nec50, marker='o', markersize=3, color='black', linestyle='None', label='mDOAS median')
ax1.bar(necrange.index, nec75-nec25, bottom=nec25, width=3, color='red', edgecolor='red', align='center', label='mDOAS 25th/75th %')
ax1.set_title('North-East Point, COSPEC + mDOAS calculated height')
#ax1.legend(loc='best', fontsize=8)

**North-East Point Contouring**

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2,ncols=1, sharex=True, figsize=(xsize, ysize))

#assumed height nea
ax0.plot(cont['obs'], marker='o', linestyle=':', color='blue', label='contouring')
ax0.bar(nearange.index, nearange['obs'], bottom=neamin['obs'], width=0.01, color='red', edgecolor='red', alpha=0.25, align='center', label='mDOAS range')
ax0.plot(nea50.index, nea50, marker='o', markersize=3, color='black', linestyle='None', label='mDOAS median')
ax0.bar(nearange.index, nea75-nea25, bottom=nea25, width=3, color='red', edgecolor='red', align='center', label='mDOAS 25th/75th %')
ax0.set_title('North-East Point, Contouring + mDOAS assumed height')
ax0.legend(loc='best', fontsize=8)

#calculated height nec
ax1.plot(cont['obs'], marker='o', linestyle=':', color='blue', label='contoring')
ax1.bar(necrange.index, necrange['obs'], bottom=necmin['obs'], width=0.01, color='red', edgecolor='red', alpha=0.25, align='center', label='mDOAS range')
ax1.plot(nec50.index, nec50, marker='o', markersize=3, color='black', linestyle='None', label='mDOAS median')
ax1.bar(necrange.index, nec75-nec25, bottom=nec25, width=3, color='red', edgecolor='red', align='center', label='mDOAS 25th/75th %')
ax1.set_title('North-East Point, Contouring + mDOAS calculated height')
#ax1.legend(loc='best', fontsize=8)

**South Rim COSPEC**

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2,ncols=1, sharex=True, figsize=(xsize, ysize))

#assumed height sra
ax0.plot(cosp['obs'], marker='o', linestyle=':', color='blue', label='cospec')
ax0.bar(srarange.index, srarange['obs'], bottom=sramin['obs'], width=0.01, color='red', edgecolor='red', alpha=0.25, align='center', label='mDOAS range')
ax0.plot(sra50.index, sra50, marker='o', markersize=3, color='black', linestyle='None', label='mDOAS median')
ax0.bar(srarange.index, sra75-sra25, bottom=sra25, width=3, color='red', edgecolor='red', align='center', label='mDOAS 25th/75th %')
ax0.set_title('South Rim, COSPEC + mDOAS assumed height')
ax0.legend(loc='best', fontsize=8)

#calculated height src
ax1.plot(cosp['obs'], marker='o', linestyle=':', color='blue', label='cospec')
ax1.bar(srcrange.index, srcrange['obs'], bottom=srcmin['obs'], width=0.01, color='red', edgecolor='red', alpha=0.25, align='center', label='mDOAS range')
ax1.plot(src50.index, src50, marker='o', markersize=3, color='black', linestyle='None', label='mDOAS median')
ax1.bar(srcrange.index, src75-src25, bottom=src25, width=3, color='red', edgecolor='red', align='center', label='mDOAS 25th/75th %')
ax1.set_title('South Rim, COSPEC + mDOAS calculated height')
#ax1.legend(loc='best', fontsize=8)

**South Rim Contouring**

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2,ncols=1, sharex=True, figsize=(xsize, ysize))

#assumed height sra
ax0.plot(cont['obs'], marker='o', linestyle=':', color='blue', label='contouring')
ax0.bar(srarange.index, srarange['obs'], bottom=sramin['obs'], width=0.01, color='red', edgecolor='red', alpha=0.25, align='center', label='mDOAS range')
ax0.plot(sra50.index, sra50, marker='o', markersize=3, color='black', linestyle='None', label='mDOAS median')
ax0.bar(srarange.index, sra75-sra25, bottom=sra25, width=3, color='red', edgecolor='red', align='center', label='mDOAS 25th/75th %')
ax0.set_title('South Rim, Contouring + mDOAS assumed height')
ax0.legend(loc='best', fontsize=8)

#calculated height src
ax1.plot(cont['obs'], marker='o', linestyle=':', color='blue', label='contouring')
ax1.bar(srcrange.index, srcrange['obs'], bottom=srcmin['obs'], width=0.01, color='red', edgecolor='red', alpha=0.25, align='center', label='mDOAS range')
ax1.plot(src50.index, src50, marker='o', markersize=3, color='black', linestyle='None', label='mDOAS median')
ax1.bar(srcrange.index, src75-src25, bottom=src25, width=3, color='red', edgecolor='red', align='center', label='mDOAS 25th/75th %')
ax1.set_title('South Rim, Contoring + mDOAS calculated height')
#ax1.legend(loc='best', fontsize=8)

Difference Histograms
--

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(nrows=2,ncols=2, sharex=True, sharey=True, figsize=(xsize, ysize))
bins=np.arange(-10,10,1)

#cosp vs nea
diffa = cospnea['obs'] - nea50
ax0.hist(diffa, bins=bins, label='assumed height')
ax0.set_ylabel('Number observations')
ax0.set_xlabel('Observation difference (kg/s)')
ax0.set_title('Difference, COSPEC - MDOAS, North-East Point')
#cosp vs nec
diffc = cospnec['obs'] - nec50
ax0.hist(diffc, bins=bins, label='calculated height')
ax0.legend(loc='best')

#cont vs nea
diffa = contnea['obs'] - nea50
ax1.hist(diffa, bins=bins, label='assumed height')
ax1.set_ylabel('Number observations')
ax1.set_xlabel('Observation difference (kg/s)')
ax1.set_title('Difference, Contouring - MDOAS, North-East Point')
#cont vs nec
diffc = contnec['obs'] - nec50
ax1.hist(diffc, bins=bins, label='calculated height')
ax1.legend(loc='best')

#cosp vs sra
diffa = cospsra['obs'] - sra50
ax2.hist(diffa, bins=bins, label='assumed height')
ax2.set_ylabel('Number observations')
ax2.set_xlabel('Observation difference (kg/s)')
ax2.set_title('Difference, COSPEC - MDOAS, South Rim')
#cosp vs src
diffc = cospsrc['obs'] - src50
ax2.hist(diffc, bins=bins, label='calculated height')
ax2.legend(loc='best')

#cont vs sra
diffa = contsra['obs'] - sra50
ax3.hist(diffa, bins=bins, label='assumed height')
ax3.set_ylabel('Number observations')
ax3.set_xlabel('Observation difference (kg/s)')
ax3.set_title('Difference, Contouring - MDOAS, South Rim')
#cont vs nec
diffc = contsrc['obs'] - src50
ax3.hist(diffc, bins=bins, label='calculated height')
ax3.legend(loc='best')

Cross plots (airborne vs MDOAS)
--

In [ ]:
xsize = 20
ysize = 10

** North-East Point**

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=1,ncols=2, sharex=True, sharey=True, figsize=(xsize, ysize))
#axes range, based on observed values, not min,max of mdoas
ax_max = max(cospnea['obs'].max(),cospnec['obs'].max(),neamean['obs'].max(),necmean['obs'].max(),contnea['obs'].max(),contnec['obs'].max())
ax_max += 1.1

#cospec
ax0.plot(cospnea['obs'], nea50, marker='o', linestyle='None', color='blue', label='assumed height')
ax0.errorbar(cospnea['obs'], nea50, xerr=cospnea['err'], yerr=0.5*nea50, linestyle='None', color='blue', label='_nolegend_')
ax0.plot(cospnec['obs'], nec50, marker='o', linestyle='None', color='red', label='calculated height')
ax0.errorbar(cospnec['obs'], nec50, xerr=cospnec['err'], yerr=0.5*nec50, linestyle='None', color='red', label='_nolegend_')
ax0.set_xlim(0,ax_max)
ax0.set_ylim(0,ax_max)
ax0.set_xlabel('COSPEC flux (kg/s)')
ax0.set_ylabel('Daily Median MDOAS flux (kg/s)')
ax0.set_title('North-East Point, COSPEC vs MDOAS Daily Median')
ax0.plot([0, ax_max], [0,ax_max], linestyle='--', color='black')
ax0.legend(loc='best')

#cospec
ax1.plot(contnea['obs'], nea50, marker='o', linestyle='None', color='blue', label='assumed height')
ax1.errorbar(contnea['obs'], nea50, xerr=contnea['err'], yerr=0.5*nea50, linestyle='None', color='blue', label='_nolegend_')
ax1.plot(contnec['obs'], nec50, marker='o', linestyle='None', color='red', label='calculated height')
ax1.errorbar(contnec['obs'], nec50, xerr=contnec['err'], yerr=0.5*nec50, linestyle='None', color='red', label='_nolegend_')

ax_max = axmax(ax1)
ax1.set_xlim(0,ax_max)
ax1.set_ylim(0,ax_max)
ax1.set_xlabel('Contoring flux (kg/s)')
ax1.set_ylabel('Daily Meadin MDOAS flux (kg/s)')
ax1.set_title('North-East Point, Contouring vs MDOAS Daily Median')
ax1.plot([0, ax_max], [0,ax_max], linestyle='--', color='black')
ax1.legend(loc='best')

**South Rim**

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=1,ncols=2, sharex=True, sharey=True, figsize=(xsize, ysize))
#axes range, based on observed values, not min,max of mdoas
ax_max = max(cospsra['obs'].max(),cospsrc['obs'].max(),sramean['obs'].max(),srcmean['obs'].max(),contsra['obs'].max(),contsrc['obs'].max())
ax_max += 1.1

#cospec
ax0.plot(cospsra['obs'], sra50, marker='o', linestyle='None', color='blue', label='assumed height')
ax0.errorbar(cospsra['obs'], sra50, xerr=cospsra['err'], yerr=0.5*sra50, linestyle='None', color='blue', label='_nolegend_')
ax0.plot(cospsrc['obs'], src50, marker='o', linestyle='None', color='red', label='calculated height')
ax0.errorbar(cospsrc['obs'], src50, xerr=cospsrc['err'], yerr=0.5*src50, linestyle='None', color='red', label='_nolegend_')
ax0.set_xlim(0,ax_max)
ax0.set_ylim(0,ax_max)
ax0.set_xlabel('COSPEC flux (kg/s)')
ax0.set_ylabel('Daily Median MDOAS flux (kg/s)')
ax0.set_title('South Rim, COSPEC vs MDOAS Daily Median')
ax0.plot([0, ax_max], [0,ax_max], linestyle='--', color='black')
ax0.legend(loc='best')

#cospec
ax1.plot(contsra['obs'], sra50, marker='o', linestyle='None', color='blue', label='assumed height')
ax1.errorbar(contsra['obs'], sra50, xerr=contsra['err'], yerr=0.5*sra50, linestyle='None', color='blue', label='_nolegend_')
ax1.plot(contsrc['obs'], src50, marker='o', linestyle='None', color='red', label='calculated height')
ax1.errorbar(contsrc['obs'], src50, xerr=contsrc['err'], yerr=0.5*src50, linestyle='None', color='red', label='_nolegend_')

ax_max = axmax(ax1)
ax1.set_xlim(0,ax_max)
ax1.set_ylim(0,ax_max)
ax1.set_xlabel('Contoring flux (kg/s)')
ax1.set_ylabel('Daily Median MDOAS flux (kg/s)')
ax1.set_title('South Rim, Contouring vs MDOAS Daily Median')
ax1.plot([0, ax_max], [0,ax_max], linestyle='--', color='black')
ax1.legend(loc='best')